In [12]:
import argparse
import os
import warnings
from pulp import *

In [13]:
cplex = getSolver('CPLEX_CMD', timeLimit=10)

# Lista de todos los nodos
Nodes = [str(i) for i in range(1,21)]

nodeData = {# NODE   Supply Demand
         "1":        [72.8 ,0],
         "2":        [8,0],
         "3":        [9,0],
         "4":        [0,0],
         "5":        [0,0],
         "6":        [0,0],
         "7":        [0,0],
         "8":        [0,0],
         "9":        [0,0],
         "10":       [0,0],
         "11":       [0,0],
         "12":       [0,0],
         "13":       [0,0],
         "14":       [0,0],
         "15":       [0,0],
         "16":       [0,0],
         "17":       [0,0],
         "18":       [0,0],
         "19":       [0,0],
         "20":       [0, 58.2]
         }


In [14]:
Arcs = [('1', '2'), ('1', '4'), ('1', '9'),
        ('2', '1'), ('2', '4'), ('2', '7'), ('2', '9'), ('2', '16'),
        ('3', '4'), ('3', '6'), ('3', '9'), ('3', '10'), ('3', '11'),
        ('4', '1'), ('4', '2'), ('4', '3'), ('4', '7'), ('4', '8'), ('4', '11'),
        ('5', '7'), ('5', '8'), ('5', '11'), ('5', '17'), ('5', '18'),
        ('6', '3'), ('6', '9'), ('6', '10'), ('6', '12'),
        ('7', '2'), ('7', '4'), ('7', '5'), ('7', '8'), ('7', '16'),
        ('8', '4'), ('8', '5'), ('8', '7'), ('8', '11'),
        ('9', '1'), ('9', '2'), ('9', '3'), ('9', '6'),
        ('10', '3'), ('10', '6'), ('10', '11'), ('10', '12'),
        ('11', '3'), ('11', '4'), ('11', '8'), ('11', '10'), ('11', '13'), ('11', '15'),
        ('12', '6'), ('12', '10'), ('12', '13'), ('12', '14'),
        ('13', '11'), ('13', '12'), ('13', '14'), ('13', '15'), ('13', '20'),
        ('14', '12'), ('14', '13'),
        ('15', '11'), ('15', '13'), ('15', '17'), ('15', '19'), ('15', '20'),
        ('16', '2'), ('16', '7'), ('16', '18'),
        ('17', '5'), ('17', '18'), ('17', '15'),
        ('18', '5'), ('18', '16'), ('18', '17'),
        ('19', '15'), ('19', '20'),
        ('20', '13'), ('20', '15'), ('20', '19')]

In [15]:
arcData = {#  ARC     Cost Min Max
        ('1', '2'):[23.2,0,4704.96],
        ('1', '4'):[22.3,0,4522.44],
        ('1', '9'):[27.3,0,5536.44],
        ('2', '1'):[23.2,0,4704.96],
        ('2', '4'):[23.3,0,4725.24],
        ('2', '7'):[11.6,0,2352.48],
        ('2', '9'):[31.9,0,6469.32],
        ('2', '16'):[13.6,0,2758.08],
        ('3', '4'):[20.7,0,4197.96],
        ('3', '6'):[32.2,0,6530.16],
        ('3', '9'):[5.9,0,1196.52],
        ('3', '10'):[5.5,0,1115.40],
        ('3', '11'):[17.3,0,3508.44],
        ('4', '1'):[22.3,0,4522.44],
        ('4', '2'):[23.3,0,4725.24],
        ('4', '3'):[20.7,0,4197.96],
        ('4', '7'):[11.8,0,2393.04],
        ('4', '8'):[8.2,0,1662.96],
        ('4', '11'):[12.7,0,2575.56],
        ('5', '7'):[8.1,0,1581.84],
        ('5', '8'):[7.8,0,3305.64],
        ('5', '11'):[16.3,0,4583.28],
        ('5', '17'):[22.6,0,6246.24],
        ('5', '18'):[30.8,0,6530.16],
        ('6', '3'):[32.2,0,5556.72],
        ('6', '9'):[27.4,0,7158.84],
        ('6', '10'):[35.3,0,8253.96],
        ('6', '12'):[40.7,0,2352.48],
        ('7', '2'):[11.6,0,2393.04],
        ('7', '4'):[11.8,0,1642.68],
        ('7', '5'):[8.1,0,932.88],
        ('7', '8'):[4.6,0,5090.28],
        ('7', '16'):[25.1,0,1662.96],
        ('8', '4'):[8.2,0,1581.84],
        ('8', '5'):[7.8,0,932.88],
        ('8', '7'):[4.6,0,2372.76],
        ('8', '11'):[11.7,0,5536.44],
        ('9', '1'):[27.3,0,6469.32],
        ('9', '2'):[31.9,0,1196.52], ('9', '3'):[5.9,0,5556.72], ('9', '6'):[27.4,0,5556.72],
        ('10', '3'):[5.1,0,1034.28], ('10', '6'):[35.3,0,7158.84], ('10', '11'):[18.1,0,3670.68], ('10', '12'):[15.6,0,3163.68],
        ('11', '3'):[17.3,0,3508.44], ('11', '4'):[12.7,0,2575.56], ('11', '8'):[11.5,0,2332.20], ('11', '10'):[18.2,0,3690.96], ('11', '13'):[27.0,0,5475.60], ('11', '15'):[18.5,0,5475.60],
        ('12', '6'):[40.7,0,3751.80], ('12', '10'):[15.6,0,8253.96], ('12', '13'):[15.4,0,3163.68], ('12', '14'):[22.4,0,3123.12],
        ('13', '11'):[27.0,0,5475.60], ('13', '12'):[15.4,0,3123.12], ('13', '14'):[25.4,0,2717.52], ('13', '15'):[25.4,0,5151.12], ('13', '20'):[25.4,0,5151.12],
        ('14', '12'):[22.4,0,4542.72], ('14', '13'):[13.4,0,2717.52],
        ('15', '11'):[18.5,0,3751.80], ('15', '13'):[18.2,0,5151.12], ('15', '17'):[10.7,0,3690.96], ('15', '19'):[7.1,0,2169.96], ('15', '20'):[0.5,0,1439.88],
        ('16', '2'):[13.6,0,2758.08], ('16', '7'):[23.0,0,4664.40], ('16', '18'):[12.1,0,2453.88],
        ('17', '5'):[22.6,0,4583.28], ('17', '18'):[32.0,0,6489.60], ('17', '15'):[18.2,0,3690.96],
        ('18', '5'):[30.8,0,6246.24], ('18', '16'):[13.4,0,2717.52], ('18', '17'):[31.2,0,6327.36],
        ('19', '15'):[10.8,0,2190.24], ('19', '20'):[14.4,0,2920.32],
        ('20', '13'):[35.9,0,7280.52], ('20', '15'):[7.9,0,1602.12], ('20', '19'):[14.4,0,2920.32]}



In [18]:
(supply, demand) = splitDict(nodeData)
(costs, mins, maxs) = splitDict(arcData)

# Crea las variables como enteros
vars = LpVariable.dicts("Ruta",Arcs,None,None,LpInteger)

# Crea los límites superior e inferior de las variables
for a in Arcs:
    vars[a].bounds(mins[a], maxs[a])

# Crea la variable 'prob' para contener los datos del problema
prob = LpProblem("Problema flujo costo minimo",LpMinimize)

# Creación de la función Objetivo
prob += lpSum([vars[a]* costs[a] for a in Arcs]), "Total costo de transporte"

# Crea todas las restricciones del problema: esto asegura que la cantidad que ingresa a cada nodo sea al menos igual a la cantidad que sale
for n in Nodes:
    prob += (supply[n]+ lpSum([vars[(i,j)] for (i,j) in Arcs if j == n]) >=
             demand[n]+ lpSum([vars[(i,j)] for (i,j) in Arcs if i == n])), "Conservación del flujo de acero en el nodo %s"%n

# Los datos del problema se escriben en un archivo .lp
prob.writeLP('Problema-flujo-costo-minimo.lp')


[Ruta_('1',_'2'),
 Ruta_('1',_'4'),
 Ruta_('1',_'9'),
 Ruta_('10',_'11'),
 Ruta_('10',_'12'),
 Ruta_('10',_'3'),
 Ruta_('10',_'6'),
 Ruta_('11',_'10'),
 Ruta_('11',_'13'),
 Ruta_('11',_'15'),
 Ruta_('11',_'3'),
 Ruta_('11',_'4'),
 Ruta_('11',_'8'),
 Ruta_('12',_'10'),
 Ruta_('12',_'13'),
 Ruta_('12',_'14'),
 Ruta_('12',_'6'),
 Ruta_('13',_'11'),
 Ruta_('13',_'12'),
 Ruta_('13',_'14'),
 Ruta_('13',_'15'),
 Ruta_('13',_'20'),
 Ruta_('14',_'12'),
 Ruta_('14',_'13'),
 Ruta_('15',_'11'),
 Ruta_('15',_'13'),
 Ruta_('15',_'17'),
 Ruta_('15',_'19'),
 Ruta_('15',_'20'),
 Ruta_('16',_'18'),
 Ruta_('16',_'2'),
 Ruta_('16',_'7'),
 Ruta_('17',_'15'),
 Ruta_('17',_'18'),
 Ruta_('17',_'5'),
 Ruta_('18',_'16'),
 Ruta_('18',_'17'),
 Ruta_('18',_'5'),
 Ruta_('19',_'15'),
 Ruta_('19',_'20'),
 Ruta_('2',_'1'),
 Ruta_('2',_'16'),
 Ruta_('2',_'4'),
 Ruta_('2',_'7'),
 Ruta_('2',_'9'),
 Ruta_('20',_'13'),
 Ruta_('20',_'15'),
 Ruta_('20',_'19'),
 Ruta_('3',_'10'),
 Ruta_('3',_'11'),
 Ruta_('3',_'4'),
 Ruta_('3

In [17]:
prob.solve(cplex)

# Se imprime el estado de la solución en la pantalla.
print("Estado:", LpStatus[prob.status])

# Cada una de las variables se imprime con su valor óptimo resuelto
for v in prob.variables():
    print(v.name, "=", v.varValue)

# El valor de la función objetivo optimizado se muestra en la pantalla
print("Total Cost of Transportation = ", value(prob.objective))

print ("\n Análisis de Sensibilidad \n Restricciones\t\t Precio Sombra \tSlack")
for name, c in prob.constraints.items():
    print (name, ":", c, "\t", c.pi, "\t\t", c.slack)


Estado: Optimal
Ruta_('1',_'2') = 0.0
Ruta_('1',_'4') = 42.0
Ruta_('1',_'9') = 0.0
Ruta_('10',_'11') = 0.0
Ruta_('10',_'12') = 0.0
Ruta_('10',_'3') = 0.0
Ruta_('10',_'6') = 0.0
Ruta_('11',_'10') = 0.0
Ruta_('11',_'13') = 0.0
Ruta_('11',_'15') = 59.0
Ruta_('11',_'3') = 0.0
Ruta_('11',_'4') = 0.0
Ruta_('11',_'8') = 0.0
Ruta_('12',_'10') = 0.0
Ruta_('12',_'13') = 0.0
Ruta_('12',_'14') = 0.0
Ruta_('12',_'6') = 0.0
Ruta_('13',_'11') = 0.0
Ruta_('13',_'12') = 0.0
Ruta_('13',_'14') = 0.0
Ruta_('13',_'15') = 0.0
Ruta_('13',_'20') = 0.0
Ruta_('14',_'12') = 0.0
Ruta_('14',_'13') = 0.0
Ruta_('15',_'11') = 0.0
Ruta_('15',_'13') = 0.0
Ruta_('15',_'17') = 0.0
Ruta_('15',_'19') = 0.0
Ruta_('15',_'20') = 59.0
Ruta_('16',_'18') = 0.0
Ruta_('16',_'2') = 0.0
Ruta_('16',_'7') = 0.0
Ruta_('17',_'15') = 0.0
Ruta_('17',_'18') = 0.0
Ruta_('17',_'5') = 0.0
Ruta_('18',_'16') = 0.0
Ruta_('18',_'17') = 0.0
Ruta_('18',_'5') = 0.0
Ruta_('19',_'15') = 0.0
Ruta_('19',_'20') = 0.0
Ruta_('2',_'1') = 0.0
Ruta_('2',_'16'